## 生成候选框 代码分析 utils/generate_anchors

In [4]:
import keras as k
import tensorflow as tf
import numpy as np


# Get all combinations of scales and ratios
scales = (32, 64, 128, 256, 512)#宽高大小
ratios = [0.5, 1, 2]#三种比例的矩形
# np.meshgrid 按x,y维度，分别扩充scales,ratios，结果维度(5,3)
scales, ratios = np.meshgrid(np.array(scales), np.array(ratios))
print("scales",scales)
print("ratios",ratios)

scales [[ 32  64 128 256 512]
 [ 32  64 128 256 512]
 [ 32  64 128 256 512]]
ratios [[0.5 0.5 0.5 0.5 0.5]
 [1.  1.  1.  1.  1. ]
 [2.  2.  2.  2.  2. ]]


In [5]:
scales = scales.flatten()#将二维降到一维(32, 64, 128, 256, 512, ...)横着循环3次
ratios = ratios.flatten()#将二维降到一维(0.5, 0.5, 0.5, 0.5, 0.5, 1, 1, ...)竖着循环5次
print("scales",scales)
print("ratios",ratios)

scales [ 32  64 128 256 512  32  64 128 256 512  32  64 128 256 512]
ratios [0.5 0.5 0.5 0.5 0.5 1.  1.  1.  1.  1.  2.  2.  2.  2.  2. ]


In [6]:
# Enumerate heights and widths from scales and ratios
# 框的宽高
heights = scales / np.sqrt(ratios)
widths = scales * np.sqrt(ratios)
print("heights",heights)
print("widths",widths)

heights [ 45.254834    90.50966799 181.01933598 362.03867197 724.07734394
  32.          64.         128.         256.         512.
  22.627417    45.254834    90.50966799 181.01933598 362.03867197]
widths [ 22.627417    45.254834    90.50966799 181.01933598 362.03867197
  32.          64.         128.         256.         512.
  45.254834    90.50966799 181.01933598 362.03867197 724.07734394]


In [8]:
# Enumerate shifts in feature space
# 每个框的中心坐标
# shape特征图宽高，np.arange生成0到某个值-1的按步长间隔的数组，feature_stride(特征步长其中一个)=[4, 8, 16, 32, 64]，anchor_stride=1
# 第一层特征缩到原图的1/4，第二层再缩到1/2(原图1/8)，第三层再缩到1/2(原图1/16)，第四层再缩到1/2(原图1/32)，第五层再缩到1/2(原图1/64)
shape = [1024/4,1024/4]
anchor_stride = 1
feature_stride = 4
shifts_y = np.arange(0, shape[0], anchor_stride) * feature_stride
shifts_x = np.arange(0, shape[1], anchor_stride) * feature_stride
print("shifts_y",shifts_y)
print("shifts_x",shifts_x)

shifts_y [   0.    4.    8.   12.   16.   20.   24.   28.   32.   36.   40.   44.
   48.   52.   56.   60.   64.   68.   72.   76.   80.   84.   88.   92.
   96.  100.  104.  108.  112.  116.  120.  124.  128.  132.  136.  140.
  144.  148.  152.  156.  160.  164.  168.  172.  176.  180.  184.  188.
  192.  196.  200.  204.  208.  212.  216.  220.  224.  228.  232.  236.
  240.  244.  248.  252.  256.  260.  264.  268.  272.  276.  280.  284.
  288.  292.  296.  300.  304.  308.  312.  316.  320.  324.  328.  332.
  336.  340.  344.  348.  352.  356.  360.  364.  368.  372.  376.  380.
  384.  388.  392.  396.  400.  404.  408.  412.  416.  420.  424.  428.
  432.  436.  440.  444.  448.  452.  456.  460.  464.  468.  472.  476.
  480.  484.  488.  492.  496.  500.  504.  508.  512.  516.  520.  524.
  528.  532.  536.  540.  544.  548.  552.  556.  560.  564.  568.  572.
  576.  580.  584.  588.  592.  596.  600.  604.  608.  612.  616.  620.
  624.  628.  632.  636.  640.  644.  648.

In [9]:
#shifts_x((1024/4),(1024/4))，扩展成二维矩阵
#shifts_x（沿X轴排列，Y轴不变），shifts_y（沿Y轴排列，X轴不变）
shifts_x, shifts_y = np.meshgrid(shifts_x, shifts_y)
print("shifts_y",shifts_y)
print("shifts_x",shifts_x)

shifts_y [[   0.    0.    0. ...    0.    0.    0.]
 [   4.    4.    4. ...    4.    4.    4.]
 [   8.    8.    8. ...    8.    8.    8.]
 ...
 [1012. 1012. 1012. ... 1012. 1012. 1012.]
 [1016. 1016. 1016. ... 1016. 1016. 1016.]
 [1020. 1020. 1020. ... 1020. 1020. 1020.]]
shifts_x [[   0.    4.    8. ... 1012. 1016. 1020.]
 [   0.    4.    8. ... 1012. 1016. 1020.]
 [   0.    4.    8. ... 1012. 1016. 1020.]
 ...
 [   0.    4.    8. ... 1012. 1016. 1020.]
 [   0.    4.    8. ... 1012. 1016. 1020.]
 [   0.    4.    8. ... 1012. 1016. 1020.]]


In [10]:
# Enumerate combinations of shifts, widths, and heights
# 生成每个坐标与宽高对应(65536=(1024/4)*(1024/4), 15)
# shifts_x,shifts_y会先由二维变成一维
box_widths, box_centers_x = np.meshgrid(widths, shifts_x)
box_heights, box_centers_y = np.meshgrid(heights, shifts_y)
print("box_widths",box_widths)
print("box_centers_x",box_centers_x)
print("box_heights",box_heights)
print("box_centers_y",box_centers_y)

box_widths [[ 22.627417    45.254834    90.50966799 ... 181.01933598 362.03867197
  724.07734394]
 [ 22.627417    45.254834    90.50966799 ... 181.01933598 362.03867197
  724.07734394]
 [ 22.627417    45.254834    90.50966799 ... 181.01933598 362.03867197
  724.07734394]
 ...
 [ 22.627417    45.254834    90.50966799 ... 181.01933598 362.03867197
  724.07734394]
 [ 22.627417    45.254834    90.50966799 ... 181.01933598 362.03867197
  724.07734394]
 [ 22.627417    45.254834    90.50966799 ... 181.01933598 362.03867197
  724.07734394]]
box_centers_x [[   0.    0.    0. ...    0.    0.    0.]
 [   4.    4.    4. ...    4.    4.    4.]
 [   8.    8.    8. ...    8.    8.    8.]
 ...
 [1012. 1012. 1012. ... 1012. 1012. 1012.]
 [1016. 1016. 1016. ... 1016. 1016. 1016.]
 [1020. 1020. 1020. ... 1020. 1020. 1020.]]
box_heights [[ 45.254834    90.50966799 181.01933598 ...  90.50966799 181.01933598
  362.03867197]
 [ 45.254834    90.50966799 181.01933598 ...  90.50966799 181.01933598
  362.0386719

In [12]:
# Reshape to get a list of (y, x) and a list of (h, w)
# 分离坐标与宽高
# np.stack后面加一个维度，数量为2，box_centers_y与box_centers_x对应维度组合
# np.reshape把(65536,15,2)转成(65536*15,2)
box_centers = np.stack(
    [box_centers_y, box_centers_x], axis=2).reshape([-1, 2])
box_sizes = np.stack([box_heights, box_widths], axis=2).reshape([-1, 2])
print("box_centers",box_centers)
print("box_sizes",box_sizes)

box_centers [[   0.    0.]
 [   0.    0.]
 [   0.    0.]
 ...
 [1020. 1020.]
 [1020. 1020.]
 [1020. 1020.]]
box_sizes [[ 45.254834    22.627417  ]
 [ 90.50966799  45.254834  ]
 [181.01933598  90.50966799]
 ...
 [ 90.50966799 181.01933598]
 [181.01933598 362.03867197]
 [362.03867197 724.07734394]]


In [19]:
# Convert to corner coordinates (y1, x1, y2, x2)
# 转成相对于左上角与右下角的框数据(y1, x1, y2, x2)
# np.concatenate按对应维度进行数据拼接，(65536*15,2), axis=1(第一维度不变，合并第二维度)，(65536*15, 4)
a1 = box_centers - 0.5 * box_sizes
print("a1",a1.shape)
boxes = np.concatenate([box_centers - 0.5 * box_sizes,
                        box_centers + 0.5 * box_sizes], axis=1)
print("boxes",boxes.shape)
print("boxes",boxes)

a1 (983040, 2)
boxes (983040, 4)
boxes [[ -22.627417    -11.3137085    22.627417     11.3137085 ]
 [ -45.254834    -22.627417     45.254834     22.627417  ]
 [ -90.50966799  -45.254834     90.50966799   45.254834  ]
 ...
 [ 974.745166    929.49033201 1065.254834   1110.50966799]
 [ 929.49033201  838.98066402 1110.50966799 1201.01933598]
 [ 838.98066402  657.96132803 1201.01933598 1382.03867197]]
